# Zadania

In [9]:
# Two-class MNIST 

from keras.datasets import mnist
import numpy as np

d1 = 5
d2 = 6

(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()

X_train = (mnist_x_train.astype('float32') / 255.).reshape((len(mnist_x_train), np.prod(mnist_x_train.shape[1:])))
y_train = mnist_y_train
X_test = (mnist_x_test.astype('float32') / 255.).reshape((len(mnist_x_test), np.prod(mnist_x_test.shape[1:])))
y_test = mnist_y_test

X_train = X_train[np.logical_or(y_train == d1, y_train == d2)]
y_train = y_train[np.logical_or(y_train == d1, y_train == d2)]
y_train[y_train==d1] = 0
y_train[y_train==d2] = 1
X_test = X_test[np.logical_or(y_test == d1, y_test == d2)]
y_test = y_test[np.logical_or(y_test == d1, y_test == d2)]
y_test[y_test==d1] = 0
y_test[y_test==d2] = 1

print X_train.shape
print X_test.shape

(11339L, 784L)
(1850L, 784L)


In [67]:
#print input.shape
#input = np.append(X_train, np.array([[1] for i in X_train]), axis=1)
#print np.append(input, [[1 for i in input[0]]],axis=0).shape

print  np.multiply([1,2,3], [1,2,3])

(11339L, 785L)
(11340L, 785L)
[1 4 9]


## Ćwiczenie 1 [5 pkt]

Uzupełnij metody forward_pass oraz backward_pass w klasach ReLU, Sigmoid i Dense. Metoda forward_pass ma przyjmować batch inputów i zwracać batch outputów. Metoda backward_pass ma przyjmować batch inputów oraz batch pochodnych cząstkowych outputów i zwracać batch pochodnych cząstkowych inputów oraz wektor (**nie batch**) pochodnych cząstkowych wag. Jeśli wagi przechowujemy w macierzy dwuwymiarowej, to możemy najpierw policzyć pochodne cząstkowe w macierzy o takim samym kształcie, a następnie np. użyć .flat.

## Ćwiczenie 2 [4 pkt]

Uzupełnij metodę _forward_pass klasy Network. Metoda ta ma przyjmować batch inputów (X) i zwracać dwie rzeczy:
* inps - lista batchów inputów dla każdej warstwy w sieci (włącznie z X); te wartości będziemy używali w metodzie _backward_pass
* output - batch outputów z sieci (czyli $\mathbf{\hat y}$); output **nie** powinien być ostatnim elementem inps.

## Ćwiczenie 3 [5 pkt]

Uzupełnij metodę _backward_pass klasy Network. Zwróć uwagę, że pochodna funkcji kosztu po neuronach ostatniej warstwy jest już liczona w metodzie _fit_on_batch. Metoda ma zwracać listę layer_grads, której elementy to wektory pochodnych cząstkowych funkcji kosztu po kolejnych warstwach (zwrócone przez metodę Layer.backward_pass). Kolejność wektorów w tej liście ma być zgodna z kolejnością warstw w sieci.

## Ćwiczenie 4 [3 pkt]
Naucz sieć neuronową z jedną warstwą ukrytą i aktywacją Sigmoid na powyższych danych (dwuklasowy MNIST). Użyj MSE jako funkcji kosztu (oznacza to regresję do numeru klasy, co jest złym pomysłem, ale póki nie mamy klasy Crossentropy musi nam to wystarczyć). Użyj GD. Reportuj loss oraz accuracy.

## Ćwiczenie 5 [3 pkt]
Uzupełnić klasę Crossentropy, wzorując się na klasie MSE.

## Ćwiczenie  6 [3 pkt]
Uzupełnić klasę Momentum, wzorując się na klasie GD. Wzory można znaleźć tutaj: http://distill.pub/2017/momentum/

## Ćwiczenie 7 [3 pkt]
Naucz sieć neuronową z jedną warstwą ukrytą. Rozważ dwa przypadki: aktywację ReLU oraz Sigmoid. Czy jest sens używać ReLU jako ostatnią warstwę? Użyj Crossentropy jako funkcji kosztu. Użyj Momentum. Reportuj loss oraz accuracy.

## Ćwiczenie 8 [6 pkt]
Vanishing gradient.

Zadanie polega na zbadaniu zjawiska *vanishing gradient* w głębokich sieciach. Należy zmodyfikować kod warstwy Dense i dodać monitorowanie **normy euklidesowej** wektora delta_weights. Każdą warstwę Dense w trenowanej sieci należy monitorować oddzielnie. Po każdym wywołaniu metody fit_on_batch każdy z monitorów powinien zapamiętać nową normę. Po nauczeniu sieci dla każdej warstwy należy narysować wykres: poziomo - numer wywołania fit_on_batch, pionowo - norma delta_weights. Im niżej znajduje się warstwa Dense, tym silniej będzie zachodziło zjawisko *vanishing gradient*.

Naucz dwuwarstwową sieć z aktywacjami Sigmoid, reportując normy delta_weights. Powtórz to dla głębszej sieci (np. 6-10 warstw).

## Ćwiczenie 9 [4 pkt]
Przetestować kod z ćwiczenia 7. (dwuwarstwowa sieć) stosując inne inicjalizacje wag w warstwach Dense. Napisać własną inicjalizację wag, która sprawi, że sieć niczego się nie nauczy (init='stupid').


In [10]:
# Warstwy
import math
import numpy as np
class Layer():

    def forward_pass(self, input):
        # return output
        raise NotImplementedError()

    def backward_pass(self, input, output_grad):
        # return input_grad, weight_grad
        raise NotImplementedError()

    def update_weights(self, delta_weights):
        pass

    def debug_grad(self, evaluate_loss):
        return None

class ReLU(Layer):

    def forward_pass(self, input):
        return np.array([ [ max(0,x) for x in y] for y in input])
        
    def backward_pass(self, input, output_grad):
        return  np.array([ np.multiply( output_grad[i],  [ 1 if x>=0 else 0 for x in input[i]]   ) for i in xrange(len(input))])    
        

class Sigmoid(Layer):

    def forward_pass(self, input):
        return np.array([ [ 1.0/(1.0+math.exp(-x)) for x in y] for y in input])
        pass

    def backward_pass(self, input, output_grad):
        return  np.array([ np.multiply( output_grad[i],  [ 1.0/(1.0+math.exp(-x)) * (1.0 - 1.0/(1.0+math.exp(-x)) )    for x in input[i]]   ) for i in xrange(len(input))])    
        

class Dense(Layer):

    def __init__(self, input_size, output_size, init = 'gaussian'):
        input_size += 1
        if init == 'zeros':
            self.weights = np.zeros((input_size, output_size))
        elif init == 'gaussian':
            np.random.seed(1)
            self.weights = np.random.normal(
                0.,
                2. / (input_size + output_size),
                (input_size, output_size)
            )
        elif init == 'aaa':
            # Ćwiczenie 8
            raise NotImplementedError()            
        else:
            raise NotImplementedError()
        self.weights = np.asmatrix(self.weights)

    def forward_pass(self, input):
        input = np.append(input, np.array([[1] for i in input]), axis=1)
        return np.dot(input, self.weights)
        #return  np.array([ np.dot(np.append(input[i], 1), self.weights)   for i in xrange(len(input))])    
        
    
    def backward_pass(self, input, output_grad):
        N = len(input)
        input = np.append(input, np.array([[1] for i in input]), axis=1)
        n = len(input[0])
        m = len(output_grad[0])
        #input_grad = np.array([ [  np.dot(output_grad[a], self.weights[b])   for b in xrange(n)]   for a in xrange(N)])    
        input_grad = np.dot(output_grad, self.weights.T)
        #print "TEMPO", output_grad.shape,  input.shape
        #weight_grad = np.dot(output_grad,input)\
        weight_grad=  np.array( [ [  np.sum( np.multiply(output_grad[:,b], input[:,a]) )  for b in xrange(m)]  for a in xrange(n) ] )  
        #print "TEMPO22", weight_grad.shape
        #return input_grad[:,1:], weight_grad.flat
        return input_grad, weight_grad.flat

    def backward_pass_2(self, input, output_grad):
        input_grad = np.dot(output_grad, self.weights.T)
        weight_grad= np.dot(input.T, output_grad)
    
        return input_grad[:,1:], weight_grad#.flat
        
    def update_weights(self, delta_weights):
        # Ćwiczenie 7 - monitorowanie normy wektora delta_weights
        #print "self.weights.shape", self.weights.shape
        #print "delta_weights", delta_weights.shape
        #print "delta_weights.reshape(self.weights.shape).shape",delta_weights.reshape(self.weights.shape).shape
        #print self.weights
        #print "---"
        #print delta_weights.reshape(self.weights.shape)
        self.weights += delta_weights.reshape(self.weights.shape)

    def debug_grad(self, evaluate_loss):
        base = evaluate_loss()
        grad = []
        for (x, y), w in np.ndenumerate(self.weights):
            self.weights[x, y] = w + 0.0001
            changed = evaluate_loss()
            grad.append(10000. * (changed - base))
            self.weights[x, y] = w
        return grad

In [11]:
assert np.allclose(ReLU().forward_pass(np.array([ [-1,-2], [1,0], [2,2] ])), np.array([[0, 0], [1, 0], [2, 2]]))
assert np.allclose(Sigmoid().forward_pass(np.array([ [0,0,0], [1,1,1], [2,2,2] ])), np.array([[ 0.5       ,  0.5       ,  0.5       ],[ 0.73105858,  0.73105858,  0.73105858],[ 0.88079708,  0.88079708,  0.88079708]]))
assert np.allclose(ReLU().backward_pass(np.array([ [-5,-3], [1.2,2], [-2,2.5] ]), np.array([ [1,-2], [1.1,0], [-2,2.7] ])), np.array([[ 0. , -0. ],[ 1.1,  0. ],[-0. ,  2.7]]) )
assert np.allclose(Sigmoid().backward_pass(np.array([ [0,0,0], [1,1,1], [2,2,2] ]), np.array([ [0,0,0], [1,1,1], [2,2,2] ])), np.array([[ 0.        ,  0.        ,  0.        ],[ 0.19661193,  0.19661193,  0.19661193], [ 0.20998717,  0.20998717,  0.20998717]]))
#print Dense(input_size=3,output_size=2).forward_pass( np.array( [  [0,0,0,0],[1,1,1,0],[2,2,2,0]  ] ) )
tmp1, tmp2= Dense(input_size=3,output_size=2).backward_pass( np.array( [  [0,0,0],[1,1,0],[2,2,0]  ] ), np.array( [  [0,0],[1,1],[2,2]  ] ) )
print tmp1, tmp2


[[ 0.          0.          0.          0.        ]
 [ 0.33752965 -0.53371346 -0.47871036  0.32786829]
 [ 0.6750593  -1.06742692 -0.95742071  0.65573658]] <numpy.flatiter object at 0x0000000003EFAA10>


In [12]:
# Optimizers

class Optimizer():

    def calculate_deltas(self, grad):
        raise NotImplementedError()

class GD(Optimizer):

    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def calculate_deltas(self, grad):
        return -self.learning_rate * grad

class Momentum(Optimizer):

    def __init__(self, alpha, beta):
        # Ćwiczenie 6
        pass
        
    def calculate_deltas(self, grad):
        # Ćwiczenie 6
        pass
        

In [46]:
# Funkcje kosztu

class Loss():

    def forward_pass(self, y, t):
        # return cost
        raise NotImplementedError()

    def backward_pass(self, y, t):
        # return y_grad
        raise NotImplementedError()

class MSE(Loss):

    def forward_pass(self, y, t):
        return np.average(0.5 * np.square(y - t))

    def backward_pass(self, y, t):
        return (y - t) / y.size

class Crossentropy(Loss):

    def forward_pass(self, y, t):
        return np.average(-1.0*np.multiply(t, np.log(y))-np.multiply(1.0 - t, np.log( 1.0 - y )) )
        
    def backward_pass(self, y, t):
        return ( -1.0*np.divide(t,y) + np.divide((1.0-y),(1.0-y)) )/ y.size


In [47]:
class Network():

    def __init__(self, loss, optimizer, metrics = []):
        self.layers = []
        self.loss = loss
        self.optimizer = optimizer
        self.metrics = metrics

    def add(self, layer):
        self.layers.append(layer)

    def fit(self, X, t, epochs, batch_size=256, print_stats=False):
        X = np.array(X)
        t = np.array(t)
        X = X.reshape(len(X), -1)
        t = t.reshape(len(t), -1)
        if X.shape[0] != t.shape[0]:
            raise ValueError("Array sizes don't match")

        for epoch in range(epochs):
            if print_stats:
                print("Epoch %d" % (epoch+1))
                print("    -> batch size: %d" % batch_size)
            rng_state = np.random.get_state()
            np.random.shuffle(X)
            np.random.set_state(rng_state)
            np.random.shuffle(t)
            pos = 0
            while pos < len(X):
                batch_X = X[pos:pos+batch_size]
                batch_t = t[pos:pos+batch_size]
                self._fit_on_batch(batch_X, batch_t)
                pos += batch_size
            if print_stats:
                _, y = self._forward_pass(X)
                l = self.loss.forward_pass(y, t)
                print("    -> loss: %f" % l)
                for m in self.metrics:
                    print("    -> %s: %f" % (m.__name__, m(y, t)))

    def predict(self, X):
        inps, out = self._forward_pass(X)
        return out

    def _fit_on_batch(self, batch_X, batch_t):
        inps, out = self._forward_pass(batch_X)
        layer_grads = self._backward_pass(
            inps,
            self.loss.backward_pass(out, batch_t)
        )
        grad = self._join(layer_grads)
        deltas = self.optimizer.calculate_deltas(grad)
        for l, d in zip(self.layers, self._split(deltas, layer_grads)):
            if not d is None:
                l.update_weights(d)

    def _join(self, grads):
        #print [g for g in grads if not g is None]
        return np.concatenate([g for g in grads if not g is None])

    def _split(self, grads, layer_grads):
        out = []
        start = 0
        for l in layer_grads:
            if l is None:
                out.append(None)
            else:
                out.append(grads[start:start+len(l)])
                start += len(l)
        return out

    def _forward_pass(self, X):
        inps = []
        output = None

        # Ćwiczenie 2
        inps.append(X)
        for layer in self.layers:
            inps.append(layer.forward_pass(inps[-1]))
            
        output = inps[-1]
        inps.pop()
        return inps, output

    def _backward_pass(self, inps, grad):
        n = len(self.layers)
        layer_grads = [None for i in xrange(n)]
        weight_grad = [None for i in xrange(n)]
        layer_grads[n-1] = grad

        # Ćwiczenie 3
        for i in xrange(n-1, 0,-1):
            input_grad, weights_grad = self.layers[i-1].backward_pass(inps[i-1], layer_grads[i]  ) 
        
            # backward_pass daje nam batch gradientów outputów podczas gdy layer_grads[n-1] 
            weight_grad[i-1] =  weights_grad
            layer_grads[i-1] =  input_grad
        return weight_grad

    def _debug_grads(self, X, t):
        layer_grads = []
        for l in self.layers:
            g = l.debug_grad(
                lambda: self.loss.forward_pass(self._forward_pass(X)[1], t)
            )
            if not g is None:
                g = np.array(np.array(g).flat)
            layer_grads.append(g)
        return layer_grads

In [ ]:
# Ćwiczenie 4
from sklearn.metrics import accuracy_score
def metric(y,t):
    return accuracy_score(np.around(y.flat), t)

network = Network(loss=Crossentropy(), optimizer=GD(learning_rate=0.01), metrics=[metric])
network.add(Dense(784,1))
network.add(Sigmoid())
network.fit(X_train,y_train, epochs= 15, print_stats=True)

Epoch 1
    -> batch size: 256
    -> loss: 0.324252
    -> metric: 0.906429
Epoch 2
    -> batch size: 256
    -> loss: 0.243941
    -> metric: 0.921951
Epoch 3
    -> batch size: 256


In [18]:
xxx = network.predict(X_test)

In [30]:
from sklearn.metrics import accuracy_score
print np.around(xxx.flat)
accuracy_score(np.around(xxx.flat), y_test)

[ 1.  1.  0. ...,  1.  0.  1.]


0.96972972972972971

In [11]:
# Ćwiczenie 7

In [12]:
# Ćwiczenie 8

In [13]:
# Ćwiczenie 9